<a href="https://colab.research.google.com/github/astromad/MyDeepLearningRepo/blob/master/generateDataKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faker

     |████████████████████████████████| 1.0MB 4.2MB/s 


In [3]:
import nltk
import random
import numpy as np
import json      
import pandas as pd
import re
from faker import Faker
import base64


def myencode(str):
    return base64.b64encode(str.encode('ascii')).decode('ascii')

def mydecode(h):
    return base64.b64decode(h.encode('ascii')).decode('ascii')

#nltk.download('punkt')
def append_sentense(record, index, sen):
    #words = nltk.word_tokenize(sen)
    #print(sen)
    words = sen.strip().split()
    #record ={'sentence_id': [], 'words': [], 'labels': []}
    for w in words:
        #print(w)
        if '#REPSTART#' in w:            
            #print(w)
            sp = re.findall(r'#REPSTART#(.+?)#REPEND#',w)[0]
            #print('hash code=',sp)
            sp = mydecode(sp)
            #print('decoded string=',sp)
            tok = re.findall(r'#REPEND#(.+?)#END#',w)[0]
            if '#NOSPLIT#'in w:
                record['sentence_idx'].append(index)
                record['word'].append(sp.strip())
                record['tag'].append(tok)
            else:
                #key = re.findall(r'#KEY=#(.+?)#END#',w)[0]
                #print('**********',mydecode(key))
                #print("string is'\''+mydecode(key)+'\''"+sp)
                for j in sp.strip().split():
                    record['sentence_idx'].append(index)
                    record['word'].append(j)
                    record['tag'].append(tok)
        else:
            record['sentence_idx'].append(index)
            record['word'].append(w)
            record['tag'].append('O')
    #print(record)

def prepare_record(data,sen_index,tok_name,split,split_key=''):
    if split :
        #record ='#REPSTART#'+myencode(' '.join(data.split(split_key)))+'#REPEND#'+tok_name+'#END##SPLIT#KEY=#\''+myencode(split_key)+'\'#END#'
        record ='#REPSTART#'+myencode(' '.join(data.split(split_key)))+'#REPEND#'+tok_name+'#END##SPLIT#'

    else:
        record ='#REPSTART#'+myencode(data)+'#REPEND#'+tok_name+'#END##NOSPLIT#'
    
    return record

def create_data(x,SSN_Dict=None,Valid=True):
    fake = Faker()
    # dictionary
    #test_data ={}
    test_data = {'sentence_idx': [], 'word': [], 'tag': []}
    #print(test_data)
    SSN_proximity = ['SSN', 'social security number', 'social', 'S S N']
    CREDIT_proximity = ['credit card', 'CCARD', 'credit', 'card number']

    for i in range(0, x):
        #test_data[i] = {}
        name = prepare_record(data=fake.name(),sen_index=i,tok_name='I-NAME',split=True,split_key=' ')
        date = prepare_record(data=fake.date(),sen_index=i,tok_name='I-DATE',split=False,split_key=' ')
        if SSN_Dict == None :
            SSN = prepare_record(data=fake.ssn(),sen_index=i,tok_name='I-SSN',split=False,split_key='-')
        else:
            if Valid:
                SSN = prepare_record(data=random.choice(SSN_Dict),sen_index=i,tok_name='I-SSN',split=False,split_key='-')
            else:
                SSN = prepare_record(data=random.choice(SSN_Dict),sen_index=i,tok_name='O',split=False,split_key='-')
        
        ccard = prepare_record(data=fake.credit_card_number(),sen_index=i,tok_name='I-CCARD',split=False,split_key=' ')
        phone = prepare_record(data=fake.phone_number(),sen_index=i,tok_name='I-PHONE',split=False,split_key=' ')

        sent = fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None)
        SSN_pword = random.choice(SSN_proximity)
        CCARD_pword = random.choice(CREDIT_proximity)
        Sentence = '{}, Her name is {}, Born on {} , Her {} is {}, Her {} is {} , and her phone number is {} '.format(sent, name,date, SSN_pword, SSN,CCARD_pword,ccard,phone)
        #Sentence = 'Her name is Radha'

        #SSN = '#REPSTART#'+myencode(' '.join(random.choice(InvalidSSN).split('-')))+'#REPEND#'+'O'+'#END#'
        #print(Sentence)
        append_sentense(test_data, i, Sentence)
    return test_data

def main():
    validSSN = []
    InvalidSSN = []
    validSSNnoSEP =[]
    with open('/content/drive/My Drive/ColabData/SSN-Valid.txt') as f:
        validSSN = f.read().splitlines()
    with open('/content/drive/My Drive/ColabData/SSN-Valid-nosep.txt') as f:
        validSSNnoSEP = f.read().splitlines()
    with open('/content/drive/My Drive/ColabData/SSN-Invalid.txt') as f:
        invalidSSN = f.read().splitlines()
    #print(validSSNnoSEP)
    
    #print(tdata)
    #print(InvalidSSN)
    tdata1 = create_data(10000,SSN_Dict=None,Valid=True)
    tdata2 = create_data(10000,SSN_Dict=validSSN,Valid=True)
    tdata3 = create_data(10000,SSN_Dict=validSSNnoSEP,Valid=True)
    tdata4 = create_data(10000,SSN_Dict=invalidSSN,Valid=False)

    #print(tdata)
    df1 = pd.DataFrame.from_dict(tdata1)
    df2 = pd.DataFrame.from_dict(tdata2)
    df3 = pd.DataFrame.from_dict(tdata3)
    df4 = pd.DataFrame.from_dict(tdata4)

    df = pd.concat([df1,df2,df3,df4])
    df.to_csv('/content/drive/My Drive/ColabData/data_keras.csv', index=False)
    df = pd.read_csv("/content/drive/My Drive/ColabData/data_keras.csv",
                 encoding="ISO-8859-1", error_bad_lines=False)
    data = df[['sentence_idx', 'word', 'tag']]

    print(data.head(60))
    lcount = df['tag'].value_counts()
    print('Labels and count')
    print(lcount)

if __name__ == '__main__' :  
    main()

    sentence_idx                word      tag
0              0                 Son        O
1              0               heavy        O
2              0             student        O
3              0                even        O
4              0             every.,        O
5              0                 Her        O
6              0                name        O
7              0                  is        O
8              0              Johnny   I-NAME
9              0              Pierce   I-NAME
10             0                Born        O
11             0                  on        O
12             0          1981-04-12   I-DATE
13             0                   ,        O
14             0                 Her        O
15             0                   S        O
16             0                   S        O
17             0                   N        O
18             0                  is        O
19             0         087-57-4161    I-SSN
20             0                 H